<a href="https://colab.research.google.com/github/gmkumar1988/Data-Analytics-Projects/blob/main/Natural_Language_Processing_Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00


In [2]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nlp
import random


def show_history(h):
    epochs_trained = len(h.history['loss'])
    plt.figure(figsize=(16, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(0, epochs_trained), h.history.get('accuracy'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_accuracy'), label='Validation')
    plt.ylim([0., 1.])
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(0, epochs_trained), h.history.get('loss'), label='Training')
    plt.plot(range(0, epochs_trained), h.history.get('val_loss'), label='Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


def show_confusion_matrix(y_true, y_pred, classes):
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y_true, y_pred, normalize='true')

    plt.figure(figsize=(8, 8))
    sp = plt.subplot(1, 1, 1)
    ctx = sp.matshow(cm)
    plt.xticks(list(range(len(classes))), labels=classes)
    plt.yticks(list(range(len(classes))), labels=classes)
    plt.colorbar(ctx)
    plt.show()


print('Using TensorFlow version', tf.__version__)

Using TensorFlow version 2.18.0


In [5]:
amazon_data = pd.read_csv('/content/amazon_reviews.csv', header=None)
amazon_data.head()

/tmp/ipython-input-2812497319.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon_data = pd.read_csv('/content/amazon_reviews.csv', header=None)


,0,1,2,3,4,5,6,7
0,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
1,032bcafa-54e0-4eb5-9e43-c86f8c5ac5e0,Garry Gordon,perfect,5,0,30.14.0.100,2025-08-04 12:33:57,30.14.0.100
2,9109695d-0b7d-48aa-a5a8-232e3978c7be,Marcia Chapman,you make it EXTREMELY IMPOSSIBLE to find and p...,1,0,30.14.0.100,2025-08-04 11:46:54,30.14.0.100
3,d329bbac-b2e2-40a8-bcec-1dea9aa35233,DominicJ Romero,Thanks for sharing,4,0,30.14.0.100,2025-08-04 10:31:01,30.14.0.100
4,c07a7bd2-79f9-4fbf-8599-f7ba1eb0e867,John Hulme,bril,5,0,30.14.0.100,2025-08-04 09:35:44,30.14.0.100


In [7]:
new_header = amazon_data.iloc[0]
amazon_data = amazon_data[1:]
amazon_data.columns = new_header
amazon_data.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
1,032bcafa-54e0-4eb5-9e43-c86f8c5ac5e0,Garry Gordon,perfect,5,0,30.14.0.100,2025-08-04 12:33:57,30.14.0.100
2,9109695d-0b7d-48aa-a5a8-232e3978c7be,Marcia Chapman,you make it EXTREMELY IMPOSSIBLE to find and p...,1,0,30.14.0.100,2025-08-04 11:46:54,30.14.0.100
3,d329bbac-b2e2-40a8-bcec-1dea9aa35233,DominicJ Romero,Thanks for sharing,4,0,30.14.0.100,2025-08-04 10:31:01,30.14.0.100
4,c07a7bd2-79f9-4fbf-8599-f7ba1eb0e867,John Hulme,bril,5,0,30.14.0.100,2025-08-04 09:35:44,30.14.0.100
5,5a600712-8e68-4334-9cfe-16f275d8ccfe,Royce Miller,there's no way to talk directly with an agent....,1,0,30.14.0.100,2025-08-04 09:30:15,30.14.0.100


In [8]:
amazon_data.columns

Index(['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'appVersion'],
      dtype='object', name=0)

In [9]:
amazon_data.shape

(77129, 8)

In [10]:
amazon_data.ndim

2

In [11]:
amazon_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77129 entries, 1 to 77129
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              77129 non-null  object
 1   userName              77123 non-null  object
 2   content               77123 non-null  object
 3   score                 77129 non-null  object
 4   thumbsUpCount         77129 non-null  object
 5   reviewCreatedVersion  68454 non-null  object
 6   at                    77129 non-null  object
 7   appVersion            68454 non-null  object
dtypes: object(8)
memory usage: 4.7+ MB


In [12]:
amazon_data.describe()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
count,77129,77123,77123,77129,77129,68454,77129,68454
unique,75594,63794,74110,10,666,254,76961,254
top,42104c7f-ba02-4f1d-8e63-755c438a1585,A Google user,Good,1,0,28.13.6.100,2025-05-14 10:22:59,28.13.6.100
freq,11,7819,294,30591,38579,2035,14,2035


In [13]:
amazon_data.head(20)

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
1,032bcafa-54e0-4eb5-9e43-c86f8c5ac5e0,Garry Gordon,perfect,5,0,30.14.0.100,2025-08-04 12:33:57,30.14.0.100
2,9109695d-0b7d-48aa-a5a8-232e3978c7be,Marcia Chapman,you make it EXTREMELY IMPOSSIBLE to find and p...,1,0,30.14.0.100,2025-08-04 11:46:54,30.14.0.100
3,d329bbac-b2e2-40a8-bcec-1dea9aa35233,DominicJ Romero,Thanks for sharing,4,0,30.14.0.100,2025-08-04 10:31:01,30.14.0.100
4,c07a7bd2-79f9-4fbf-8599-f7ba1eb0e867,John Hulme,bril,5,0,30.14.0.100,2025-08-04 09:35:44,30.14.0.100
5,5a600712-8e68-4334-9cfe-16f275d8ccfe,Royce Miller,there's no way to talk directly with an agent....,1,0,30.14.0.100,2025-08-04 09:30:15,30.14.0.100
6,bd9f353b-cfc2-439c-8310-5da159b40bd6,Capt. jack sparrow,best of the best,5,0,30.14.0.100,2025-08-04 09:26:54,30.14.0.100
7,0e94af65-c11a-4c82-84e5-8e9d5b61a494,Nicholas Carter,"Amazon rules, sometimes......",5,0,30.14.0.100,2025-08-04 09:09:27,30.14.0.100
8,be9a7603-5cf6-4fc1-ac29-8ff569a1ecc0,Tyler Kelley,19 items i paid to have delivered the next day...,2,0,30.14.0.100,2025-08-04 08:49:13,30.14.0.100
9,4941ec19-8365-4736-a832-3d33a6ef0f28,Amrit Raythere223,"it's really bad and poor service, no one respo...",1,0,NaN,2025-08-04 08:32:48,NaN
10,db88245a-2642-4630-a580-1880c2757cf0,Haseeb Ahmad,no 1 prodect you send me insha alla,5,0,30.14.0.100,2025-08-04 07:46:11,30.14.0.100


In [14]:
amazon_data_train = amazon_data.sample(frac=0.8, random_state=25)

In [15]:
print("No. of training samples: ", amazon_data_train.shape[0])

No. of training samples:  61703


In [16]:
amazon_data_train.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
27493,8924ddb3-1477-4be9-accb-48208f8c3fdc,Phil Wells,The app is ok and to be honest the customer se...,2,19,28.4.0.100,2024-03-03 19:23:52,28.4.0.100
29435,94ad1051-f3e4-4457-99d8-07f90ece3cce,Jason B,Pretty much done. As Amazon continues to add f...,1,1,28.3.0.100,2024-02-12 05:32:24,28.3.0.100
67392,5c6b23a7-2612-4f4a-9263-32ca155cdf65,Dorthy Cotner,Amazon has always delivered on time. Returns a...,5,0,26.15.0.100,2023-08-26 15:56:26,26.15.0.100
64456,ebf3efe7-e9d5-4059-b044-b556d7103053,Tünde Toth,The crashes I'm unable to open it since yester...,1,0,22.6.0.100,2021-03-23 08:29:10,22.6.0.100
63281,a9377dcc-0865-428b-bbe0-6ecfbeb20aa9,moo dubose,Everytime I search for any book the first opti...,2,0,24.22.0.100,2022-12-31 05:38:05,24.22.0.100


In [17]:
amazon_data_test = amazon_data.drop(amazon_data_train.index)

In [18]:
print("No. of testing samples: ", amazon_data_test.shape[0])

No. of testing samples:  15426


In [19]:
amazon_data_test.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
21,e00281cc-7fa8-4286-ba8b-203035a8f514,Michael Bidwell,Shes my mistress for christmas,5,0,30.14.0.100,2025-08-04 02:23:37,30.14.0.100
22,24847c0a-935c-4a3b-a6c6-5fc156ed4d08,Natalia Zvonitskaia,"buy a lot ,very usefull",5,0,30.14.0.100,2025-08-04 01:14:15,30.14.0.100
23,ef4f2bc4-8054-4f37-a32a-2488177480c3,T Rabbitmom,very easy & convenient,5,0,30.14.0.100,2025-08-04 00:36:44,30.14.0.100
30,b1ada782-c457-4687-9a25-1b2e247fcfe9,Linda Dugre,Easy ordering and many choices,5,0,30.14.0.100,2025-08-03 11:59:42,30.14.0.100
50,6c0412ea-a414-422d-9ce7-11db4743ee14,Tamika Warren,good morning,5,0,30.11.0.100,2025-08-03 03:06:46,30.11.0.100


In [20]:
amazon_data_train['content'].head()

,content
27493,The app is ok and to be honest the customer se...
29435,Pretty much done. As Amazon continues to add f...
67392,Amazon has always delivered on time. Returns a...
64456,The crashes I'm unable to open it since yester...
63281,Everytime I search for any book the first opti...


In [21]:
amazon_data_test['content'].head()

,content
21,Shes my mistress for christmas
22,"buy a lot ,very usefull"
23,very easy & convenient
30,Easy ordering and many choices
50,good morning


In [22]:
def get_review(data):
  reviews = [x['text'] for x in data]
  return reviews

In [26]:
train_amazon_reviews = amazon_data_train['content']
test_amazon_reviews = amazon_data_test['content']

In [30]:
train_amazon_reviews.iloc[20]

"I HATE your new filter. It's slow. It's Not intuitive. It's medevil. It doesn't produce results any better than if i did it on my own. Your search results typically are not even relevant to what I'm looking for. I could pay the neighbor kid five bucks to walk to the store and buy what I need faster. I used to spend 75% of my shopping budget with Amazon but now, I don't even spend 10%. Not the Amazon cares because they aren't responding or doing anything to change."

In [31]:
test_amazon_reviews.iloc[30]

'Enjoy using app'